In [ ]:
import os
import numpy as np
import nibabel as nib
from skimage.filters import threshold_otsu

# 文件夹路径，用于保存处理后的图像
folders = {
    'ct_high': r'G:\PD_L2\ct_high',
    'ct_low': r'G:\PD_L2\ct_low',
    'pet_high': r'G:\PD_L2\pet_high',
    'pet_low': r'G:\PD_L2\pet_low',
    'high_high': r'G:\PD_L2\high_high',
    'low_low': r'G:\PD_L2\low_low',
    'high_low': r'G:\PD_L2\high_low',
    'low_high': r'G:\PD_L2\low_high',
    'combined_roi': r'G:\PD_L2\combined_roi'
}

# 确保目标文件夹存在
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

def process_image_and_roi(ct_path, pet_path, roi_path):
    base_filename = os.path.splitext(os.path.basename(ct_path))[0]

    ct_img = nib.load(ct_path)
    pet_img = nib.load(pet_path)
    roi_img = nib.load(roi_path)

    ct_data = ct_img.get_fdata()
    pet_data = pet_img.get_fdata()
    roi_mask = roi_img.get_fdata().astype(bool)

    # Apply the ROI mask
    ct_data_roi = ct_data[roi_mask]
    pet_data_roi = pet_data[roi_mask]

    # 分别对CT和PET图像的ROI区域应用Otsu算法
    ct_threshold = threshold_otsu(ct_data_roi)
    pet_threshold = threshold_otsu(pet_data_roi)

    # Based on the threshold, classify the ROI in original data
    ct_high = np.zeros(ct_data.shape, dtype=np.uint8)
    ct_low = np.zeros(ct_data.shape, dtype=np.uint8)
    pet_high = np.zeros(pet_data.shape, dtype=np.uint8)
    pet_low = np.zeros(pet_data.shape, dtype=np.uint8)

    ct_high[roi_mask] = ct_data[roi_mask] > ct_threshold
    ct_low[roi_mask] = ct_data[roi_mask] <= ct_threshold
    pet_high[roi_mask] = pet_data[roi_mask] > pet_threshold
    pet_low[roi_mask] = pet_data[roi_mask] <= pet_threshold

    # 创建复合ROI
    combined_roi = np.zeros(ct_data.shape, dtype=np.uint8)
    combined_roi[roi_mask] = (ct_high[roi_mask] & pet_high[roi_mask]) * 1 + \
                             (ct_low[roi_mask] & pet_low[roi_mask]) * 2 + \
                             (ct_high[roi_mask] & pet_low[roi_mask]) * 3 + \
                             (ct_low[roi_mask] & pet_high[roi_mask]) * 4

    # 保存每种情况的图像和复合ROI
    for label, data in [('ct_high', ct_high), ('ct_low', ct_low),
                        ('pet_high', pet_high), ('pet_low', pet_low),
                        ('high_high', ct_high & pet_high), ('low_low', ct_low & pet_low),
                        ('high_low', ct_high & pet_low), ('low_high', ct_low & pet_high),
                        ('combined_roi', combined_roi)]:
        save_path = os.path.join(folders[label], f"{base_filename}_{label}.nii")
        nib.save(nib.Nifti1Image(data, affine=ct_img.affine), save_path)

    print(f"Processed and saved: {base_filename}")

# 用实际路径替换下面的占位符来处理和保存一个图像集
ct_path = r'G:\PD_L1\images_CT\CT_1.nii.gz'
pet_path = r'G:\PD_L1\images_CT\CT_1.nii.gz'
roi_path = r'G:\PD_L1\masks\ROI_1.nii.gz'
process_image_and_roi(ct_path, pet_path, roi_path)